In [3]:
import ollama
from smolagents import CodeAgent, LiteLLMModel, DuckDuckGoSearchTool, ToolCallingAgent, tool

import os
import re
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify

oc = ollama.Client("http://localhost:11434")

CACHE_DIR = "/users/formation/irtn7prtnc/LLM_Valdom/Cache"
RAG_DIR = "/users/formation/irtn7prtnc/LLM_Valdom/RAG"

os.makedirs(CACHE_DIR, exist_ok = True)
os.makedirs(RAG_DIR, exist_ok = True)

model = LiteLLMModel(
    model_id = "ollama/qwen2.5-coder:32b", #['deepseek-r1:32b', 'qwen2.5-coder:32b', 'llama3.1:8b', 'mistral-nemo:latest', 'mistral:latest']
    api_base = "http://localhost:11434/api/generate",
    num_ctx = 24000
    )

## Configuration des Outils

In [ ]:
import requests
import re
import json
from markdownify import markdownify
from requests.exceptions import RequestException
from bs4 import BeautifulSoup

@tool
def visit_webpage(url: str) -> str:
    """Visits a webpage at the given URL and returns its content as a markdown string.

    Args:
        url: The URL of the webpage to visit.

    Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()
        
        # Convert the HTML content to Markdown
        markdown_content = markdownify(response.text).strip()
        
        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
        return markdown_content
    
    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
   
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

#####################################################################################################
    
@tool
def download_file(url: str) -> str:
    """Download a file from a give URL to a target destination.

    Args:
        url: The URL of the file to download.
        
    Returns:
        The downloaded file path, or an error message if file download failed
    """
    try:
        local_filename = os.path.join(CACHE_DIR, url.split('/')[-1])
        
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            
            with open(local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        return f"Fichier téléchargé avec succès : {local_filename}"
    
    except Exception as e:
        return f"Erreur lors du téléchargement : {str(e)}"


#####################################################################################################
 
def unzipp_file(path: str) -> str:
    """ ""
    

## Configuration des Agents


In [9]:

web_agent = ToolCallingAgent(
    tools = [DuckDuckGoSearchTool(), visit_webpage, download_file],
    model = model,
    add_base_tools = True,
    max_steps = 10,
    name="Web_agent",
    description = "An agent that is able to extrat information and navigate inside a given website with the aim to access and download pertinent data inside this webside regarding the request")

#####################################################################################################

data_agent = ToolCallingAgent(
    tools=[],
    model=model,
    add_base_tools = False,
    max_steps = 10,
    name = "Data_agent",
    description = "An useless agent for now, but you can discuss with him about weather if you want")

#####################################################################################################

infograpic_agent = ToolCallingAgent(
    tools=[],
    model=model,
    add_base_tools = False,
    max_steps = 10,
    name = "Info_agent",
    description = "An useless agent for now, but you can discuss with him about weather if you want")

#####################################################################################################

manager_agent = CodeAgent(
    tools = [],
    model = model,
    managed_agents = [web_agent, data_agent],
    additional_authorized_imports = ["time", "numpy", "pandas"],
    planning_interval = 3,
    verbosity_level = 2,
    #add_base_tools = True
    max_steps = 5
)



## Utilisation de l'agent


In [ ]:
agent_output = manager_agent.run('discuss with others agents')

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ discuss with others agents                                                                                      │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5-coder:32b ───────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
1. Define the term "preparatory survey" by requesting information from Web_agent.
2. Request Web_agent to gather guidelines on the structure of a comprehensive preparatory survey from professional 
organizations or industry standards documents.
3. Ask Web_agent to collect information on the types of facts that should be included in a preparatory survey based
on best practices guides and case studies.
4. Review the gathered information with Data_agent to ensure all relevant details are considered, even if it's just
for discussion purposes.
5. Determine the relevance of each fact collected by analyzing how they contribute to completing the task 
successfully.
6. Organize the facts in a logical hierarchy to create an effective and structured preparatory survey.
7. Present the findings and organized structure of the preparatory survey to other agents for feedback and further 
input.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
### Thought:                                                                                                       
The task is to discuss with other agents to gather information about what constitutes a preparatory survey, its    
structure, and the types of facts it should include. I will start by asking Web_agent to provide definitions and   
guidelines related to preparatory surveys.                                                                         
                                                                                                                   
### Code:                                                                                                          
```py                                                                                                              
Web_agent(task="Provide a definition of 'preparatory survey' from academic or professional sources.")              
Web_agent(task="Gather guidelines on the structure of a comprehensive preparatory survey from professional         
organizations or industry standards documents.")                                                                   
Web_agent(task="Collect information on the types of facts that should be included in a preparatory survey based on 
best practices guides and case studies.")                                                                          
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  Web_agent(task="Provide a definition of 'preparatory survey' from academic or professional sources.")            
  Web_agent(task="Gather guidelines on the structure of a comprehensive preparatory survey from professional       
  organizations or industry standards documents.")                                                                 
  Web_agent(task="Collect information on the types of facts that should be included in a preparatory survey based  
  on best practices guides and case studies.")                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

Final answer: ### 1. Task outcome (short version):
Failed to retrieve specific guidelines on the structure of a comprehensive preparatory survey from professional 
organizations or industry standards documents due to persistent connection errors with the search tool.

### 2. Task outcome (extremely detailed version):
The task aimed to gather structured guidelines on how to conduct a comprehensive preparatory survey by consulting 
official documents and industry standards from relevant professional organizations. Multiple attempts were made 
using the web_search tool with various queries including searching for PDFs and academic sources. However, all 
attempts resulted in DuckDuckGoSearchException errors indicating connection issues (Connection refused, os error 
111). Despite retrying with different query variations, no successful search results could be obtained.

### 3. Additional context (if relevant):
The persistent failure to connect to the web_search tool suggests there might be an underlying issue with network 
connectivity or access to the service. It is recommended to check the network settings or try a different internet 
connection if available. Alternatively, other methods of obtaining the required information, such as directly 
visiting known professional organization websites or using alternative search engines via visit_webpage, could be 
explored.

Given these challenges, further action is needed to resolve the connectivity issue before the task can be 
successfully completed.

[Step 6: Duration 8.01 seconds| Input tokens: 19,620 | Output tokens: 429]

╭────────────────────────────────────────────── New run - Web_agent ──────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'Web_agent'.                                                                       │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Collect information on the types of facts that should be included in a preparatory survey based on best         │
│ practices guides and case studies.                                                                              │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5-coder:32b ───────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': 'best practices for preparatory survey facts'}             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯